<img src="https://unskript.com/assets/favicon.png" alt="unSkript.com" width="100" height="100"/> 
<h1> unSkript Runbooks </h1>
<div class="alert alert-block alert-success">
    <b> This runbook demonstrates How to Secure Publicly accessible Amazon RDS Snapshot using unSkript legos.</b>
</div>

<br>

<center><h2>Secure Publicly accessible Amazon RDS Snapshot</h2></center>

# Steps Overview
   1. List all the manual database snapshots in the given region.
   2. Get publicly accessible DB snapshots in RDS.
   3. Modify the publicly accessible DB snapshots in RDS to private.

Here we will use unSkript AWS Filter All Manual Databse Snapshots Lego. This lego take region: str as input. This inputs is used to findout all Manual Database Snapshots.

In [3]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
from unskript.connectors.aws import aws_get_paginator
import pprint


from beartype import beartype
@beartype
def aws_get_manual_database_snapshots_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_get_manual_database_snapshots(handle, region: str) -> List:
    """aws_get_manual_snapshots List all the manual database snapshots.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type region: string
        :param region: Region for database.

        :rtype: List of manual database snapshots.
    """

    ec2Client = handle.client('rds', region_name=region)
    result = []
    try:
        response = aws_get_paginator(ec2Client, "describe_db_snapshots","DBSnapshots",
                                     SnapshotType='manual')
        for snapshot in response:
            result.append(snapshot['DBSnapshotIdentifier'])
    except Exception as error:
        result.append(error)

    return result


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "region": "Region"
    }''')
task.configure(outputName="ManualSnapshots")
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_manual_database_snapshots, lego_printer=aws_get_manual_database_snapshots_printer, hdl=hdl, args=args)

Here we will use unSkript AWS Get Publicly Accessible DB Snapshots in RDS Lego. This lego take db_snapshot_identifier: list and region: str as input. This inputs is used to findout publicly accessible Database Snapshots from the given list.

In [4]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
import pprint

from beartype import beartype
@beartype
def aws_get_publicly_accessible_db_snapshots_printer(output):
    if output is None:
        return
    pprint.pprint(output)



@beartype
def aws_get_publicly_accessible_db_snapshots(handle, db_snapshot_identifier: list, region: str) -> List:
    """aws_get_publicly_accessible_db_snapshots lists of publicly accessible db_snapshot_identifier.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type db_snapshot_identifier: List
        :param db_snapshot_identifier: DB Snapshot Idntifier of RDS.

        :type region: string
        :param region: Region of the RDS.

        :rtype: List with publicly accessible Snapshots Identifier in RDS.
    """

    ec2Client = handle.client('rds', region_name=region)
    result = []
    try:
        for identifier in db_snapshot_identifier:
            response = ec2Client.describe_db_snapshot_attributes(
                DBSnapshotIdentifier=identifier)
            db_attribute = response["DBSnapshotAttributesResult"]
            for value in db_attribute['DBSnapshotAttributes']:
                if "all" in value["AttributeValues"]:
                    result.append(db_attribute['DBSnapshotIdentifier'])

    except Exception as error:
        result.append(error)

    return result


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "db_snapshot_identifier": "ManualSnapshots",
    "region": "Region"
    }''')
task.configure(outputName="PublicSnapshots")

(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_get_publicly_accessible_db_snapshots, lego_printer=aws_get_publicly_accessible_db_snapshots_printer, hdl=hdl, args=args)

Here we will use unSkript AWS Modify Publicly Accessible RDS Snapshots Lego. This lego take db_snapshot_identifier: str, region: str as input. This inputs is used to modify publicly accessible RDS snapshots to private.

In [ ]:
##  Copyright (c) 2021 unSkript, Inc
##  All rights reserved.
##
from typing import List, Dict
from pydantic import BaseModel, Field
import pprint

from beartype import beartype
@beartype
def aws_modify_public_db_snapshots_printer(output):
    if output is None:
        return
    pprint.pprint(output)


@beartype
def aws_modify_public_db_snapshots(handle, db_snapshot_identifier: str, region: str) -> List:
    """aws_modify_public_db_snapshots lists of publicly accessible DB Snapshot Idntifier Info.

        :type handle: object
        :param handle: Object returned from task.validate(...).

        :type db_snapshot_identifier: string
        :param db_snapshot_identifier: DB Snapshot Idntifier of RDS.

        :type region: string
        :param region: Region of the RDS.

        :rtype: List with Dict of DB Snapshot Idntifier Info.
    """

    ec2Client = handle.client('rds', region_name=region)
    result = []
    try:
        response = ec2Client.modify_db_snapshot_attribute(
            DBSnapshotIdentifier=db_snapshot_identifier,
            AttributeName='restore',
            ValuesToRemove=['all'])
        result.append(response)
    except Exception as error:
        result.append(error)

    return result


task = Task(Workflow())
task.configure(continueOnError=True)
task.configure(inputParamsJson='''{
    "region": "Region",
    "db_snapshot_identifier": "iter_item"
    }''')
task.configure(iterJson='''{
    "iter_enabled": true,
    "iter_list_is_const": false,
    "iter_list": "PublicSnapshots",
    "iter_parameter": "db_snapshot_identifier"
    }''')
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(aws_modify_public_db_snapshots, lego_printer=aws_modify_public_db_snapshots_printer, hdl=hdl, args=args)

In this Runbook, we demonstrated the use of unSkript's AWS legos to modify. This runbook help to get the all manual snapshots from RDS then filter publicly accessible snapshots from given list and modify publicly accessible RDS snapshots to private. To view the full platform capabilities of unSkript please visit https://unskript.com